# exo sur les dates

# 0) imports

In [45]:
from datetime import datetime
from datetime import timedelta
import random
import pandas as pd

# 1) Création d'un dataset synthétique

documentation en ligne => être après la parenthèse de la fonction, et faire "MAJ + TAB"

## 1.1) création d'une fonction pour générer de la data 

In [88]:
def creer_une_ligne():
    """
    Renvoie un dictionnaire avec deux dates.
    """
    # création d'une commande
    annee         = 2019 
    mois          = random.randint(1,12)
    jour          = random.randint(1,28)
    heure         = random.randint(0,23)   
    minutes       = random.randint(0,59)
    date_commande = datetime(annee, mois, jour, 
                             heure, minutes)
    # création d'un délais
    delais_jour   = random.randint(0, 2)
    delais_heure   = random.randint(3, 8)
    delais_minutes = random.randint(10, 100)
    delais         = timedelta(days    = delais_jour, 
                               hours   = delais_heure, 
                               minutes = delais_minutes)

    date_envoi_depuis_entrepot = date_commande + delais
    
    return {
        "date_commande" : date_commande,
        "date_envoi_depuis_entrepot" : date_envoi_depuis_entrepot
    }

## 1.2) création d'une dataFrame

In [89]:
nb_lignes_voulu = 1000
df = pd.DataFrame([creer_une_ligne() for i in range(nb_lignes_voulu)])

# 2) Features Engineering

## 2.1) extraction du jour de la semaine

In [96]:
df["isoWeekDay"] = df.date_commande.map(lambda date: date.isoweekday())
#df["isoWeekDay"]

## 2.2) one hot encoding

In [95]:
dummies = pd.get_dummies(df["isoWeekDay"], prefix="isoWeekDay")
#dummies

## 2.3) concaténer à la DF originale

In [119]:
df2 = pd.concat([df,dummies ], axis=1)
df2.columns

Index(['date_commande', 'date_envoi_depuis_entrepot', 'isoWeekDay',
       'isoWeekDay_1', 'isoWeekDay_2', 'isoWeekDay_3', 'isoWeekDay_4',
       'isoWeekDay_5', 'isoWeekDay_6', 'isoWeekDay_7'],
      dtype='object')

## 2.4) récupération des heures 

In [127]:
df2["hour"]  = df2.date_commande.map(lambda date :date.hour)
df2["month"] = df2.date_commande.map(lambda date :date.month)

# 3) déterminer le nombre de commande depuis le début de la journée

In [130]:
df2["jour_du_mois"] = df2.date_commande.map(lambda date: date.day)

In [135]:
df2["date_calendaire"] = df2.date_commande.map(lambda d: d.date())

# méthode naïve (et je sais pas si on fera mieux :-/)

In [142]:
from collections import defaultdict

In [149]:
ligne

(0, date_commande                 2019-02-13 00:14:00
 date_envoi_depuis_entrepot    2019-02-14 07:39:00
 isoWeekDay                                      3
 isoWeekDay_1                                    0
 isoWeekDay_2                                    0
 isoWeekDay_3                                    1
 isoWeekDay_4                                    0
 isoWeekDay_5                                    0
 isoWeekDay_6                                    0
 isoWeekDay_7                                    0
 hour                                            0
 month                                           2
 jour_du_mois                                   13
 date_calendaire                        2019-02-13
 Name: 0, dtype: object)

In [160]:
result = []
for index,data  in df2.iterrows():
    # 1) récupération de data
    heure_et_date_commande   = data["date_commande"]
    aujourdhui               = data["date_calendaire"]
    heure_commande           = heure_et_date_commande.hour 
    # 2) filtre les données => uniquement les lignes de la même date
    filtre_sur_la_date       = df2.date_calendaire == aujourdhui 
    sub_df                   = df2[filtre_sur_la_date]
    # 3) filtre sur l'heure => prendre les lignes à une heure inférieure
    filtre_sur_heure = sub_df.date_commande < heure_et_date_commande
    # 4) résultat
    nb_commande_precedentes = sum(filtre_sur_heure)
    # 5) affichage
    r = { "heure_et_date_commande"  : heure_et_date_commande,
          "nb_commande_precedentes" : nb_commande_precedentes}
    result.append(r)

In [161]:
df3 = pd.DataFrame(result, index=range(len(result)))

# Joindre df2 et df3 pour rajouter la nouvelle colonne "nb_commande_precedentes" dans df2

In [196]:
# on met en index les colonnes sur lesquelles on veut joindre
df4 = df3.set_index("heure_et_date_commande")
df5 = df2.set_index("date_commande")

In [194]:

df6 = df5.join(df4)

In [198]:
df7 = df6.reset_index()

In [200]:
df7.pivot_table(index=["date_commande", "nb_commande_precedentes"])

,,hour,isoWeekDay,isoWeekDay_1,isoWeekDay_2,isoWeekDay_3,isoWeekDay_4,isoWeekDay_5,isoWeekDay_6,isoWeekDay_7,jour_du_mois,month
date_commande,nb_commande_precedentes,,,,,,,,,,,
2019-01-01 04:09:00,0,4,2,0,1,0,0,0,0,0,1,1
2019-01-01 05:40:00,1,5,2,0,1,0,0,0,0,0,1,1
2019-01-01 13:11:00,2,13,2,0,1,0,0,0,0,0,1,1
2019-01-01 14:31:00,3,14,2,0,1,0,0,0,0,0,1,1
2019-01-02 06:06:00,0,6,3,0,0,1,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27 09:32:00,1,9,5,0,0,0,0,1,0,0,27,12
2019-12-27 12:06:00,2,12,5,0,0,0,0,1,0,0,27,12
2019-12-27 17:53:00,3,17,5,0,0,0,0,1,0,0,27,12


In [163]:
!git add simulation_envoi_colis.ipynb

In [164]:
!git commit -m "pour suivre"

[master 358a55a] pour suivre
 1 file changed, 608 insertions(+), 62 deletions(-)


In [165]:
!git push origin master 

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 3.28 KiB | 3.28 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/romainjouin/esilv_5a.git
   10e2a5c..358a55a  master -> master


In [162]:
df3.sort_values("heure_et_date_commande")

,heure_et_date_commande,nb_commande_precedentes
375,2019-01-01 04:09:00,0
914,2019-01-01 05:40:00,1
208,2019-01-01 13:11:00,2
388,2019-01-01 14:31:00,3
991,2019-01-02 06:06:00,0
...,...,...
572,2019-12-27 09:32:00,1
158,2019-12-27 12:06:00,2
137,2019-12-27 17:53:00,3
37,2019-12-28 13:29:00,0


In [147]:
heure_commande

0

In [148]:
heure_et_date_commande

Timestamp('2019-02-13 00:14:00')

In [136]:
df2.groupby("date_calendaire").count()

,date_commande,date_envoi_depuis_entrepot,isoWeekDay,isoWeekDay_1,isoWeekDay_2,isoWeekDay_3,isoWeekDay_4,isoWeekDay_5,isoWeekDay_6,isoWeekDay_7,hour,month,jour_du_mois
date_calendaire,,,,,,,,,,,,,
2019-01-01,4,4,4,4,4,4,4,4,4,4,4,4,4
2019-01-02,2,2,2,2,2,2,2,2,2,2,2,2,2
2019-01-03,5,5,5,5,5,5,5,5,5,5,5,5,5
2019-01-05,1,1,1,1,1,1,1,1,1,1,1,1,1
2019-01-06,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,1,1,1,1,1,1,1,1,1,1,1,1,1
2019-12-24,2,2,2,2,2,2,2,2,2,2,2,2,2
2019-12-26,3,3,3,3,3,3,3,3,3,3,3,3,3
